<h1>Actividad Clasificador de topicos</h1>
<h2>Martes 10 de Agosto</h2>
<ol>
<li>Elegir dos topicos</li>
<li>Obtener al menos 50 frases para cada topico</li>
<li>Definir un set con almenos 25 frases de cada tópico</li>
<li>Definir el código para entrenar un modelo de SGD o Naive Bayes</li>
</ol>



In [ ]:
import pandas as np